In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.contingency_tables import StratifiedTable, Table2x2

# Read data from CSV file
df = pd.read_csv('relativerisk.csv')

In [ ]:
from tabulate import tabulate
import pandas as pd

# Misalnya, df adalah data Anda
# Ganti nilai 0 dan 1 pada kolom 'married' dan 'healthy' sebelum membuat crosstab
df['married'] = df['married'].replace({0: 'no', 1: 'yes'})
df['healthy'] = df['healthy'].replace({0: 'no', 1: 'yes'})

# Buat crosstab antara married dan healthy
crosstab = pd.crosstab(df['married'], df['healthy'], margins=True)

print("Crosstab between married and healthy:")
print(tabulate(crosstab, headers='keys', tablefmt='grid'))

# Hitung persentase dalam married
crosstab_perc = crosstab.div(crosstab['All'], axis=0) * 100
print("\nPercentages (%):")
print(tabulate(crosstab_perc, headers='keys', tablefmt='grid'))

Crosstab between married and healthy:
+-----------+------+-------+-------+
| married   |   no |   yes |   All |
+===========+======+=======+=======+
| no        |  192 |  1104 |  1296 |
+-----------+------+-------+-------+
| yes       |  167 |  1537 |  1704 |
+-----------+------+-------+-------+
| All       |  359 |  2641 |  3000 |
+-----------+------+-------+-------+

Percentages within married:
+-----------+----------+---------+-------+
| married   |       no |     yes |   All |
+===========+==========+=========+=======+
| no        | 14.8148  | 85.1852 |   100 |
+-----------+----------+---------+-------+
| yes       |  9.80047 | 90.1995 |   100 |
+-----------+----------+---------+-------+
| All       | 11.9667  | 88.0333 |   100 |
+-----------+----------+---------+-------+


In [19]:
import numpy as np
from statsmodels.stats.contingency_tables import Table2x2

# Get counts for 2x2 table using "no" and "yes" labels
unmarried_unhealthy = crosstab.loc["no", "no"]
unmarried_healthy = crosstab.loc["no", "yes"]
married_unhealthy = crosstab.loc["yes", "no"]
married_healthy = crosstab.loc["yes", "yes"]

# Construct contingency table
table = np.array([
    [unmarried_unhealthy, unmarried_healthy],
    [married_unhealthy, married_healthy]
])

# Create 2x2 table object
table2x2 = Table2x2(table)

# Calculate odds ratio and confidence interval
oddsratio = table2x2.oddsratio
oddsratio_confint = table2x2.oddsratio_confint()
print(f"\nOdds Ratio (no/yes): {oddsratio}")
print(f"95% Confidence Interval for Odds Ratio:  lower bound: {oddsratio_confint[0]}")
print(f"\t\t\t\t\t upper bound: {oddsratio_confint[1]}")


Odds Ratio (no/yes): 1.6006248372819578
95% Confidence Interval for Odds Ratio:  lower bound: 1.2828176176769495
					 upper bound: 1.9971661087438233


In [36]:
import math

# Get counts for "healthy = no" cohort
unmarried_total_no = unmarried_unhealthy + unmarried_healthy
married_total_no = married_unhealthy + married_healthy

# Probability of being unhealthy in unmarried and married groups (healthy = no)
prob_unmarried_unhealthy_no = unmarried_unhealthy / unmarried_total_no
prob_married_unhealthy_no = married_unhealthy / married_total_no

# Calculate relative risk for "healthy = no"
relative_risk_no = prob_unmarried_unhealthy_no / prob_married_unhealthy_no
print(f"\nRelative Risk (healthy = no): {relative_risk_no}")

# Standard error and 95% confidence interval for relative risk (healthy = no)
se_relative_risk_no = math.sqrt(
    (1 - prob_unmarried_unhealthy_no) / (unmarried_total_no * prob_unmarried_unhealthy_no) +
    (1 - prob_married_unhealthy_no) / (married_total_no * prob_married_unhealthy_no)
)

ci_lower_no = relative_risk_no * math.exp(-1.96 * se_relative_risk_no)
ci_upper_no = relative_risk_no * math.exp(1.96 * se_relative_risk_no)

print(f"95% Confidence Interval for Relative Risk (healthy = no):") 
print(f"lower bound: {ci_lower_no}")
print(f"upper bound: {ci_upper_yes}")

# Get counts for "healthy = yes" cohort
unmarried_total_yes = unmarried_healthy + unmarried_unhealthy
married_total_yes = married_healthy + married_unhealthy

# Probability of being healthy in unmarried and married groups (healthy = yes)
prob_unmarried_healthy_yes = unmarried_healthy / unmarried_total_yes
prob_married_healthy_yes = married_healthy / married_total_yes

# Calculate relative risk for "healthy = yes"
relative_risk_yes = prob_unmarried_healthy_yes / prob_married_healthy_yes
print(f"\nRelative Risk (healthy = yes): {relative_risk_yes}")

# Standard error and 95% confidence interval for relative risk (healthy = yes)
se_relative_risk_yes = math.sqrt(
    (1 - prob_unmarried_healthy_yes) / (unmarried_total_yes * prob_unmarried_healthy_yes) +
    (1 - prob_married_healthy_yes) / (married_total_yes * prob_married_healthy_yes)
)

ci_lower_yes = relative_risk_yes * math.exp(-1.96 * se_relative_risk_yes)
ci_upper_yes = relative_risk_yes * math.exp(1.96 * se_relative_risk_yes)

print(f"95% Confidence Interval for Relative Risk (healthy = yes)")
print(f"lower bound: {ci_lower_no}")
print(f"upper bound: {ci_upper_yes}")



Relative Risk (healthy = no): 1.511643379906853
95% Confidence Interval for Relative Risk (healthy = no):
lower bound: 1.2445729778426209
upper bound: 0.9708142557095893

Relative Risk (healthy = yes): 0.9444082989951565
95% Confidence Interval for Relative Risk (healthy = yes)
lower bound: 1.2445729778426209
upper bound: 0.9708142557095893


In [5]:
# Adjust for ageclass
ageclasses = sorted(df['ageclass'].unique())
tables = []

print("\nOdds Ratios in each ageclass:")
for age in ageclasses:
    df_age = df[df['ageclass'] == age]
    ct = pd.crosstab(df_age['married'], df_age['healthy'])
    if ct.shape == (2, 2):
        table_age = np.array([
            [ct.loc[0, 0], ct.loc[0, 1]],
            [ct.loc[1, 0], ct.loc[1, 1]]
        ])
        tables.append(table_age)
        table2x2_age = Table2x2(table_age)
        or_age = table2x2_age.oddsratio
        or_ci_age = table2x2_age.oddsratio_confint()
        print(f"Ageclass {age}: Odds Ratio = {or_age}, 95% CI = {or_ci_age}")
    else:
        print(f"Ageclass {age} does not have a full 2x2 table and will be skipped.")


Odds Ratios in each ageclass:
Ageclass 1.0: Odds Ratio = 2.324856439704676, 95% CI = (1.5105171467757021, 3.578216557669361)
Ageclass 2.0: Odds Ratio = 1.7349474219810042, 95% CI = (1.2090900247150256, 2.4895107026856724)
Ageclass 3.0: Odds Ratio = 2.3517705589698568, 95% CI = (1.2537682076774148, 4.411361468706528)
Ageclass 4.0: Odds Ratio = 1.145278450363196, 95% CI = (0.7029006744745528, 1.8660712338152807)


In [6]:
# Create StratifiedTable
mh_table = StratifiedTable(tables)

# Mantel-Haenszel common odds ratio
mh_or = mh_table.oddsratio_pooled
mh_or_confint = mh_table.oddsratio_pooled_confint()
print(f"\nMantel-Haenszel common Odds Ratio: {mh_or}")
print(f"95% Confidence Interval: {mh_or_confint}")


Mantel-Haenszel common Odds Ratio: 1.780649273613681
95% Confidence Interval: (1.4217473900860704, 2.230151331896575)


In [7]:
import pandas as pd
import numpy as np
import math
from statsmodels.stats.contingency_tables import Table2x2
from tabulate import tabulate  # Import tabulate

# Load your data
df = pd.read_csv('relativerisk.csv')

# Crosstab between married and healthy
crosstab = pd.crosstab(df['married'], df['healthy'], margins=True)

# Get counts for 2x2 table
unmarried_unhealthy = crosstab.loc[0, 0]
unmarried_healthy = crosstab.loc[0, 1]
married_unhealthy = crosstab.loc[1, 0]
married_healthy = crosstab.loc[1, 1]

# Construct contingency table
table = np.array([
    [unmarried_unhealthy, unmarried_healthy],
    [married_unhealthy, married_healthy]
])

# Create 2x2 table object
table2x2 = Table2x2(table)

# Calculate odds ratio and confidence interval
oddsratio = table2x2.oddsratio
oddsratio_confint = table2x2.oddsratio_confint()
oddsratio_ci_lower, oddsratio_ci_upper = oddsratio_confint

# Calculate relative risk manually
unmarried_total = unmarried_unhealthy + unmarried_healthy
married_total = married_unhealthy + married_healthy

# Probability of being unhealthy in unmarried and married groups
prob_unmarried_unhealthy = unmarried_unhealthy / unmarried_total
prob_married_unhealthy = married_unhealthy / married_total

# Calculate relative risk
relative_risk = prob_unmarried_unhealthy / prob_married_unhealthy

# Calculate standard error for relative risk
se_relative_risk = math.sqrt(
    (1 - prob_unmarried_unhealthy) / (unmarried_total * prob_unmarried_unhealthy) +
    (1 - prob_married_unhealthy) / (married_total * prob_married_unhealthy)
)

# Calculate 95% confidence interval for relative risk
ci_lower = relative_risk * math.exp(-1.96 * se_relative_risk)
ci_upper = relative_risk * math.exp(1.96 * se_relative_risk)

# Save all results to a dictionary
results = {
    "Odds Ratio": [oddsratio],
    "Odds Ratio CI Lower": [oddsratio_ci_lower],
    "Odds Ratio CI Upper": [oddsratio_ci_upper],
    "Relative Risk": [relative_risk],
    "Relative Risk CI Lower": [ci_lower],
    "Relative Risk CI Upper": [ci_upper]
}

# Convert dictionary to DataFrame
results_df = pd.DataFrame(results)

# Display DataFrame with grid lines in the terminal
print("Calculated Results:")
print(tabulate(results_df, headers='keys', tablefmt='grid'))

# Save to CSV
results_df.to_csv("analysis_results.csv", index=False)
print("Results saved to 'analysis_results.csv'")


Calculated Results:
+----+--------------+-----------------------+-----------------------+-----------------+--------------------------+--------------------------+
|    |   Odds Ratio |   Odds Ratio CI Lower |   Odds Ratio CI Upper |   Relative Risk |   Relative Risk CI Lower |   Relative Risk CI Upper |
+====+==============+=======================+=======================+=================+==========================+==========================+
|  0 |      1.60062 |               1.28282 |               1.99717 |         1.51164 |                  1.24457 |                  1.83602 |
+----+--------------+-----------------------+-----------------------+-----------------+--------------------------+--------------------------+
Results saved to 'analysis_results.csv'
